## Crypto-Portfolio Opmization by @Albercej

### Finding top crypto portfolios with Tensorflow and Matrix calculus

Albercej August2019 (Updated Nov 2020 to TF 2.3

https://medium.com/jordi-moraleda/crypto-portfolio-optimization-with-python-and-tensorflow-an-approach-aa504578c799

### Libraries
So let’s create a blank notebook and start with our optimizer. First, we need to import the following libraries:
import json
import requests
import pandas as pd
import numpy as np
import tensorflow as tf

### import matplotlib.pyplot as plt
json and requests are self-explanatory. pandas is very useful to work with data series and data analysis. numpy is the swiss army for scientific computing. We will mainly use it for working with matrices. tensorflow will be used to optimize the weights of our portfolio and matplotlib lets us visualize data on our notebook.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
# import matplotlib.pyplot as plt

C:\Users\alber\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\alber\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\alber\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


### Fetching historic data
To start working with data, we need to retrieve it from an API endpoint. In our case, we will use CryptoCompare. Copy this code in the Colab and run it. Let’s get some data and arrange it:


In [3]:
import json
from bs4 import BeautifulSoup
import requests

#my_key="6c425eb2-ae22-4454-9563-2bf7c3362dec"

#url = "https://pro-api.coinmarketcap.com/v1/ticker/"
#response = requests.get(url)
#soup = BeautifulSoup(response.content, "html.parser")
#dic = json.loads(soup.prettify())

In [25]:
 
# create an empty DataFrame
#df = pd.DataFrame(columns=["Ticker", "MarketCap"])
 
#for i in range(len(dic)):
#    df.loc[len(df)] = [dic[i]['symbol'], dic[i]['market_cap_usd']]

# we sort values by MarketCap 
#df.sort_values(by=['MarketCap'])
# apply conversion to numeric as 'df' contains lots of 'None' string as values
#df.MarketCap = pd.to_numeric(df.MarketCap)

# we decide min capitalization
#P = df[df.MarketCap > 20e7]
#print(P, end="\n\n")

#portfolio = list(P.Ticker)

In [26]:
#coins = []
# Select Top 10 Cryptocurrency Markets by volumen
#crypto = [e[0] for e in P.Ticker[0:10]]
#coins = portfolio[0:7]
#print(coins)

Note that in addition of fetching and parsing JSON data, we are using Pandas to turn the time column into a Python date and index by it. This allows us to filter rows by any boolean criteria and deal with date conditions very easily.
In a REPL environment, variables are displayed by just typing them. In the code above, we populated the coin_history dictionary with Pandas data frames. So let’s add a new code block in Colab, type an expression like coin_history['BTC'] and hit Enter:

In [160]:
# GLOBAL

coins = ["BTC", "ETH", "LTC", "XLM", "XRP", "ANT", "MANA", "TRX", "USDT"]
days_ago_to_fetch = 400  # see also filter_history_by_date()
coin_history = {}
hist_length = 0
average_returns = {}
cumulative_returns = {}

def fetch_history(coin):
  endpoint_url = "https://min-api.cryptocompare.com/data/histoday?fsym={}&tsym=USD&limit={:d}&aggregate=1".format(coin, days_ago_to_fetch)
  res = requests.get(endpoint_url)
  #his = pd.DataFrame(json.loads(res.content)['Data'])
  h = (json.loads(res.content))#['Data'])
  dic = pd.json_normalize(h, "Data")
  hist = pd.DataFrame(dic)
  #hist = index_history(hist)
  #hist = filter_history_by_date(hist)
  return hist

def fetch_all():
  for coin in coins:
    coin_history[coin] = fetch_history(coin)

#---
def index_history(hist):
  # index by date so we can easily filter by a given timeframe
  hist = hist.set_index('time')
  hist.index = pd.to_datetime(hist.index, unit='s')
  return hist

#def filter_history_by_date(hist):
#  result = hist[hist.index.year >= 2017]
  # result = result[result.index.day == 1] # every first of month, etc.
#  return result
#---
fetch_all()

hist_length = len(coin_history[coins[0]])

coin_history["BTC"]

,time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol
0,1570924800,8498.59,8194.16,8316.66,20190.99,1.688970e+08,8296.34,direct,
1,1571011200,8414.24,8228.06,8296.34,17507.78,1.460155e+08,8363.94,direct,
2,1571097600,8439.31,8116.92,8363.94,24899.11,2.056746e+08,8173.48,direct,
3,1571184000,8193.41,7945.77,8173.48,34102.92,2.746703e+08,8014.39,direct,
4,1571270400,8145.50,7948.95,8014.39,21657.02,1.751729e+08,8082.46,direct,
...,...,...,...,...,...,...,...,...,...
396,1605139200,16348.99,15476.25,15707.08,67322.57,1.074746e+09,16306.53,direct,
397,1605225600,16482.62,15975.80,16306.53,42676.28,6.943206e+08,16332.85,direct,
398,1605312000,16334.29,15720.61,16332.85,28331.05,4.526577e+08,16075.95,direct,
399,1605398400,16156.82,15796.03,16075.95,17966.53,2.869123e+08,15964.70,direct,


### Returns
Next, we need to add a couple of columns with values derived from the current columns: the (daily) return and the excess return.
Image for post
The return of an asset is the ratio between the price variation over the initial price.
### Image for post
The excess return over a certain period is the difference between the average return and the actual return.
In a normal scenario, we would need to write a loop and compute values for every individual row. However, Pandas allows to perform operations with the whole array with one-liners like below:

In [161]:
# Calculate returns and excess returns

def add_all_returns():
  for coin in coins:
    hist = coin_history[coin]
    hist['return'] = (hist['close']-hist['open']) / hist['open']
    average = hist["return"].mean()
    average_returns[coin] = average
    cumulative_returns[coin] = (hist["return"] + 1).prod() - 1
    hist['excess_return'] = hist['return'] - average
    coin_history[coin] = hist

add_all_returns()

#display data
cumulative_returns

{'BTC': 0.966785945319395,
 'ETH': 1.5411115599887162,
 'LTC': 0.2664256785948149,
 'XLM': 0.3321317384892686,
 'XRP': 0.018681318681317283,
 'ANT': 2.078841840257673,
 'MANA': 4.625557974004227,
 'TRX': 0.5962500000000035,
 'USDT': 8.881784197001252e-16}

### Several things just happened here:

We added the **return** and **excess_return** columns to every row of our historic data frames
We stored the **average return** of every asset
We stored the **cumulative return** for every historic we have

Now, average_returns and cumulative_returns contain the following values:

In [183]:
average_returns, cumulative_returns

({'BTC': 0.002467048010052626,
  'ETH': 0.0036106446387064485,
  'LTC': 0.0017823464171151469,
  'XLM': 0.001845924483911792,
  'XRP': 0.0008703636925237263,
  'ANT': 0.006589319642796081,
  'MANA': 0.007103791762892127,
  'TRX': 0.002584777167483989,
  'USDT': 2.0325050483854895e-06},
 {'BTC': 0.966785945319395,
  'ETH': 1.5411115599887162,
  'LTC': 0.2664256785948149,
  'XLM': 0.3321317384892686,
  'XRP': 0.018681318681317283,
  'ANT': 2.078841840257673,
  'MANA': 4.625557974004227,
  'TRX': 0.5962500000000035,
  'USDT': 8.881784197001252e-16})

Now it’s time to combine the excess returns into the n × k Excess Return Matrix X, where n is the number of observations (days, weeks, months, …) and k is the number of assets in out portfolio.
Image for post

### Excess Return Matrix

Arranging the excess return columns into a matrix can be achieved in just a few lines of Python:


In [162]:
# Excess matrix

excess_matrix = np.zeros((hist_length, len(coins)))

for i in range(0, hist_length):
    for idx, coin in enumerate(coins):
        excess_matrix[i][idx] = coin_history[coin].iloc[i]['excess_return']

The first line creates an n × k matrix and the loops assign the corresponding values to each coin.
However, if we print the values of the matrix we will get the raw content of the numpy array, which is not very meaningful.

In [163]:
excess_matrix

array([[-4.91033654e-03,  4.49776367e-03,  1.64454103e-02, ...,
        -5.85183558e-03,  2.42902228e-02, -2.03250505e-06],
       [ 5.68112335e-03,  2.71297696e-02,  4.95285387e-03, ...,
         2.60315630e-02,  2.35868601e-02, -2.03250505e-06],
       [-2.52386126e-02, -3.77100567e-02, -4.10429098e-02, ...,
        -8.61665107e-03, -6.36761176e-02, -1.00003650e-03],
       ...,
       [-1.81960849e-02, -3.68272446e-02, -3.31745211e-02, ...,
        -5.10888897e-02, -1.04495039e-02, -2.03250505e-06],
       [-9.38732333e-03, -3.07672288e-02, -2.57372548e-02, ...,
        -2.13086943e-02,  9.82404759e-04,  9.96968494e-04],
       [ 2.21115535e-02,  1.34681273e-02,  1.17845620e-01, ...,
        -1.52710978e-02,  6.10400640e-03, -2.03250505e-06]])

Let’s wrap the matrix into a Pandas DataFrame, set an appropriate index (dates) and column titles (coin names):

In [164]:
# Display
pretty_matrix = pd.DataFrame(excess_matrix).copy()
pretty_matrix.columns = coins
pretty_matrix.index = coin_history[coins[0]].index
pretty_matrix

,BTC,ETH,LTC,XLM,XRP,ANT,MANA,TRX,USDT
0,-0.004910,0.004498,0.016445,0.005200,0.017078,-0.010916,-0.005852,0.024290,-0.000002
1,0.005681,0.027130,0.004953,0.076091,0.070738,-0.000299,0.026032,0.023587,-0.000002
2,-0.025239,-0.037710,-0.041043,-0.036110,-0.032099,-0.024773,-0.008617,-0.063676,-0.001000
3,-0.021931,-0.035538,-0.041782,-0.036544,-0.017162,-0.028814,-0.015589,-0.027222,0.000997
4,0.006026,0.010451,0.046761,0.054339,0.063259,-0.068742,-0.026358,0.011016,-0.000002
...,...,...,...,...,...,...,...,...,...
396,0.035697,-0.006046,0.022152,0.000391,-0.005554,-0.111661,-0.017342,-0.007770,-0.001000
397,-0.000853,0.026836,0.082579,0.021462,0.042267,0.086084,-0.006262,0.017062,-0.000002
398,-0.018196,-0.036827,-0.033175,-0.019171,0.008528,-0.064182,-0.051089,-0.010450,-0.000002
399,-0.009387,-0.030767,-0.025737,-0.013435,0.003599,-0.027568,-0.021309,0.000982,0.000997


### Risk Modeling

There are many approaches that can be used to optimize a portfolio. In the present article we will analyze the variance and covariance of individual assets in order to minimize the global risk.
To this end, we will use our Excess Return Matrix to compute the Variance-covariance Matrix Σ from it.
Image for post

### Variance-covariance Matrix
Computing it is as simple as can be:

In [165]:
# Variance co-variance matrix

product_matrix = np.matmul(excess_matrix.transpose(), excess_matrix)
var_covar_matrix = product_matrix / hist_length

In [166]:
# Display
pretty_matrix = pd.DataFrame(var_covar_matrix).copy()
pretty_matrix.columns = coins
pretty_matrix.index = coins
pretty_matrix

,BTC,ETH,LTC,XLM,XRP,ANT,MANA,TRX,USDT
BTC,0.001450,0.001600,0.001570,0.001298,0.001219,0.001858,0.001424,0.001360,-0.000021
ETH,0.001600,0.002394,0.002079,0.001829,0.001702,0.002595,0.001783,0.001947,-0.000025
LTC,0.001570,0.002079,0.002291,0.001761,0.001640,0.002360,0.001672,0.001802,-0.000024
XLM,0.001298,0.001829,0.001761,0.002201,0.001547,0.002136,0.001655,0.001651,-0.000022
XRP,0.001219,0.001702,0.001640,0.001547,0.001574,0.001856,0.001379,0.001521,-0.000019
ANT,0.001858,0.002595,0.002360,0.002136,0.001856,0.007217,0.001931,0.001958,-0.000038
MANA,0.001424,0.001783,0.001672,0.001655,0.001379,0.001931,0.005626,0.001670,-0.000034
TRX,0.001360,0.001947,0.001802,0.001651,0.001521,0.001958,0.001670,0.002652,-0.000022
USDT,-0.000021,-0.000025,-0.000024,-0.000022,-0.000019,-0.000038,-0.000034,-0.000022,0.000004


In this matrix:
Image for post

cov x, y is the covariance between asset X and asset Y

When x = y the value is the variance of the asset
Before we can jump into the actual portfolio optimization, our next target is the Correlation Matrix, where every item is defined like:
Image for post

The correlation between assets X and Y is their covariance divided by the product of their standard deviations.
We already have cov(X, Y) stored in var_covar_matrix so we need a k × k matrix with the products of each standard deviation.
Let’s compute the individual standard deviations:

In [167]:
# Standard Deviation

std_deviations = np.zeros((len(coins), 1))

for idx, coin in enumerate(coins):
  std_deviations[idx][0] = np.std(coin_history[coin]['return'])

In [168]:
# Display
pretty_matrix = pd.DataFrame(std_deviations).copy()
pretty_matrix.columns = ["Std_Dev"]
pretty_matrix.index = coins
pretty_matrix

,Std_Dev
BTC,0.038077
ETH,0.048925
LTC,0.047863
XLM,0.046913
XRP,0.039667
ANT,0.084951
MANA,0.075004
TRX,0.051494
USDT,0.002020


To generate the matrix with the standard deviation products, we multiply the above by its transpose.

In [169]:
# Std Deviation products matrix
sdev_product_matrix = np.matmul(std_deviations, std_deviations.transpose())

In [170]:
# Display
pretty_matrix = pd.DataFrame(sdev_product_matrix).copy()
pretty_matrix.columns = coins
pretty_matrix.index = coins
pretty_matrix

,BTC,ETH,LTC,XLM,XRP,ANT,MANA,TRX,USDT
BTC,0.001450,0.001863,0.001822,0.001786,0.001510,0.003235,0.002856,0.001961,0.000077
ETH,0.001863,0.002394,0.002342,0.002295,0.001941,0.004156,0.003670,0.002519,0.000099
LTC,0.001822,0.002342,0.002291,0.002245,0.001899,0.004066,0.003590,0.002465,0.000097
XLM,0.001786,0.002295,0.002245,0.002201,0.001861,0.003985,0.003519,0.002416,0.000095
XRP,0.001510,0.001941,0.001899,0.001861,0.001574,0.003370,0.002975,0.002043,0.000080
ANT,0.003235,0.004156,0.004066,0.003985,0.003370,0.007217,0.006372,0.004374,0.000172
MANA,0.002856,0.003670,0.003590,0.003519,0.002975,0.006372,0.005626,0.003862,0.000152
TRX,0.001961,0.002519,0.002465,0.002416,0.002043,0.004374,0.003862,0.002652,0.000104
USDT,0.000077,0.000099,0.000097,0.000095,0.000080,0.000172,0.000152,0.000104,0.000004


So now, we can finally compute the Correlation Matrix, as we defined before.

In [171]:
# Correlation matrix
correlation_matrix = var_covar_matrix / sdev_product_matrix

In [172]:
# Display
pretty_matrix = pd.DataFrame(correlation_matrix).copy()
pretty_matrix.columns = coins
pretty_matrix.index = coins
pretty_matrix

,BTC,ETH,LTC,XLM,XRP,ANT,MANA,TRX,USDT
BTC,1.000000,0.858908,0.861309,0.726535,0.806955,0.574328,0.498643,0.693528,-0.279232
ETH,0.858908,1.000000,0.887714,0.797054,0.877227,0.624294,0.485911,0.772867,-0.255902
LTC,0.861309,0.887714,1.000000,0.784436,0.863701,0.580462,0.465747,0.731089,-0.247525
XLM,0.726535,0.797054,0.784436,1.000000,0.831525,0.536024,0.470301,0.683596,-0.228761
XRP,0.806955,0.877227,0.863701,0.831525,1.000000,0.550905,0.463590,0.744847,-0.236638
ANT,0.574328,0.624294,0.580462,0.536024,0.550905,1.000000,0.303103,0.447622,-0.219874
MANA,0.498643,0.485911,0.465747,0.470301,0.463590,0.303103,1.000000,0.432394,-0.224309
TRX,0.693528,0.772867,0.731089,0.683596,0.744847,0.447622,0.432394,1.000000,-0.207456
USDT,-0.279232,-0.255902,-0.247525,-0.228761,-0.236638,-0.219874,-0.224309,-0.207456,1.000000


In the Correlation Matrix:

The correlation of an asset’s returns with itself is always 1
Correlation values range from –1 to 1
Values tending to 1 mean that two random variables tend to have linear relationship
Correlation values tending to –1 (anticorrelation) mean that two assets tend to have opposite behaviors
Correlation values of 0 mean that two random variables are independent

## Portfolio optimization
Given the average return and the variance of our assets, now it’s time to decide how much money is allocated in each one.
At this point, we would like to find a combination of investments that minimizes the global variance of the portfolio.
Image for post

The weights array is the output we aim to get from our portfolio optimizer. The weight of every asset can range from 0 to 1, and the overall sum must be 1.
Given the weights array, we can define the weighted standard deviation as:
Image for post

So the global variance of our portfolio can now be defined as:
Image for post

Where W is a 1 × k matrix with the weighted standard deviations , C is the Correlation Matrix described above and the result is a 1 × 1 matrix with the global portfolio variance.
This is the value that we want to minimize, but how can we do it?
We could define functions that computed the global variance for given weight arrays, explore all the possible candidates and rate them.
However, finding the absolute minimal value for an equation with k variables is an NP problem. The amount of calculations would grow exponentially with k if we attempted to evaluate every possible solution. Waiting 10³⁰ centuries to get an answer doesn’t look like an appealing scenario, does it?
So the best alternative in our hands is to use Machine Learning to explore a diverse subset of the search space for us, and let it explore variants of branches with potential to perform better than their siblings.
Tensorflow
This is where Tensorflow comes into play. TensorFlow is an opensource Machine Learning framework originally developed by Google.
The main difference with most libraries is that, instead of performing operations directly, it provides a set of methods to describe them, so that training and optimization can be performed on them. Operations like tf.multiply(x, y) will not return the actual result, but an operation that can be run in a session and evaluated, eventually.
Sessions are made of a graph of interrelated operations and variables, and they can be spawn into CPU’s, GPU’s and the specialized TPU units. This allows us to describe operations on variables (instead of numeric values) and let the framework minimize the target value for us.
To get started with Tensorflow, take a good tutorial, play with it and come back when you are ready.
So, let’s go back to Colab and code a first approach to the problem:


### Let’s drill the function down, step by step.
The first block defines the mathematical operations that produce the global volatility. coin_weights is a k × 1 variable array with an equal amount allocated by default and from it, we get the weighted standard deviations array:
coin_weights = tf.Variable(np.full((len(coins), 1), 1.0 / len(coins)))
weighted_std_devs = tf.multiply(coin_weights, std_deviations)
With it, we can describe the matrix multiplication, in three steps:
product_1 = tf.transpose(weighted_std_devs)
product_2 = tf.matmul(product_1, correlation_matrix)
portfolio_variance = tf.matmul(product_2, weighted_std_devs)
portfolio_volatility = tf.sqrt(portfolio_variance)
Remember that these instructions are not computing anything yet. They create a graph of operations that will be executed when we run them in a session.
Running portfolio_volatility would trigger the dependent portfolio_variance tensor, which would use weighted_std_devs and trigger the dependent product_2 , etc.
Next, we initialize the variables and define what optimizer should be used to generate solutions in each training step:
init = tf.global_variables_initializer()
train_step = tf.train.GradientDescentOptimizer(learn_rate).minimize(portfolio_volatility)
Finally, we create a session, run the variable initializer operation and train the optimizer with the operation we just defined.
with tf.Session() as sess:
    sess.run(init)
    for i in range(steps):
      sess.run(train_step)
Note that train_step depends on .minimize(portfolio_volatility) , which depends on the matrix multiplications we defined before. These operations will be evaluated at each step, right after the optimizer populates the coin_weights variable with new values.

In [177]:
# Optimize weights to minimize volatility

#tf.compat.v1.disable_eager_execution()

def minimize_volatility():

  tf.compat.v1.disable_eager_execution()
  # Define the model
  # Portfolio Volatility = Sqrt (Transpose (Wt.SD) * Correlation Matrix * Wt. SD)

  coin_weights = tf.Variable(np.full((len(coins), 1), 1.0 / len(coins))) # our variables
  weighted_std_devs = tf.multiply(coin_weights, std_deviations)

  product_1 = tf.transpose(weighted_std_devs)
  product_2 = tf.matmul(product_1, correlation_matrix)
  
  portfolio_variance = tf.matmul(product_2, weighted_std_devs)
  portfolio_volatility = tf.sqrt(tf.reduce_sum(portfolio_variance))

# No constraits

  # Run
  learn_rate = 0.01
  steps = 5000
  
  init = tf.compat.v1.global_variables_initializer()

  # Training using Gradient Descent to minimize variance
  train_step = tf.compat.v1.train.GradientDescentOptimizer(learn_rate, use_locking=False).minimize(portfolio_volatility)

  with tf.compat.v1.Session() as sess:
    sess.run(init)
    for i in range(steps):
      sess.run(train_step)
      if i % 1000 == 0 :
        print("[round {:d}]".format(i))
        print("Weights", coin_weights.eval())
        print("Volatility: {:.2f}%".format(portfolio_volatility.eval() * 100))
        print("")
        
    return coin_weights.eval()

weights = minimize_volatility()

pretty_weights1 = pd.DataFrame(weights * 100, index = coins, columns = ["Min Volatility Non Constratits Weight %"])
pretty_weights1

[round 0]
Weights [[0.11077536]
 [0.11065691]
 [0.11067843]
 [0.11070964]
 [0.1107564 ]
 [0.11048642]
 [0.11062258]
 [0.11069587]
 [0.11111684]]
Volatility: 3.87%

[round 1000]
Weights [[ 1.02313708e-03]
 [-1.92246306e-05]
 [-1.11357173e-04]
 [ 9.05780324e-05]
 [-5.47523873e-05]
 [ 2.06975112e-04]
 [ 2.95776084e-04]
 [ 1.48566517e-05]
 [ 9.99968775e-02]]
Volatility: 0.02%

[round 2000]
Weights [[ 8.14132839e-04]
 [-1.36724946e-06]
 [-8.14396009e-05]
 [ 7.67343125e-05]
 [-6.35500103e-05]
 [ 1.66457745e-04]
 [ 2.39100681e-04]
 [ 1.12444563e-05]
 [ 8.07591189e-02]]
Volatility: 0.02%

[round 3000]
Weights [[ 6.20195426e-04]
 [-1.04143476e-06]
 [-6.20395409e-05]
 [ 5.84552091e-05]
 [-4.84117200e-05]
 [ 1.26805277e-04]
 [ 1.82143701e-04]
 [ 8.56587117e-06]
 [ 6.15212164e-02]]
Volatility: 0.01%

[round 4000]
Weights [[ 4.26258117e-04]
 [-7.15774415e-07]
 [-4.26395565e-05]
 [ 4.01760579e-05]
 [-3.32732034e-05]
 [ 8.71528169e-05]
 [ 1.25186720e-04]
 [ 5.88729288e-06]
 [ 4.22833138e-02]]
Volatil

,Min Volatility Non Constratits Weight %
BTC,0.039343
ETH,0.017207
LTC,0.013672
XLM,0.017640
XRP,0.011229
ANT,0.031507
MANA,0.026585
TRX,0.015867
USDT,2.644630


No big surprise so far. We start with 1 / k (around 14%) of each asset allocated but the optimizer quickly learns that the way to minimize volatility is by… not allocating anything at all. Eventually the weights would converge to zero. Ta da!
We obviously have to force values to range from 0 to 1, and make their sum be equal to 1.
Constraints
In Tensorflow we can’t assign values to variables like var = val. Values can be declared via tf.constant(...) , as a tf.Variable(...) with a default value or be assigned via my_tensor.assign(...) , which creates an operation that needs to be run.
Let’s add the coin_weights constraints and run them after each training step:


In [178]:
# Optimize weights to minimize volatility

def minimize_volatility():

  # Define the model
  # Portfolio Volatility = Sqrt (Transpose (Wt.SD) * Correlation Matrix * Wt. SD)

  coin_weights = tf.Variable(np.full((len(coins), 1), 1.0 / len(coins))) # our variables
  weighted_std_devs = tf.multiply(coin_weights, std_deviations)

  product_1 = tf.transpose(weighted_std_devs)
  product_2 = tf.matmul(product_1, correlation_matrix)
  
  portfolio_variance = tf.matmul(product_2, weighted_std_devs)
  portfolio_volatility = tf.sqrt(tf.reduce_sum(portfolio_variance))

  # Constraints: sum([0..1, 0..1, ...]) = 1

  lower_than_zero = tf.greater( np.float64(0), coin_weights )
  zero_minimum_op = coin_weights.assign( tf.where (lower_than_zero, tf.zeros_like(coin_weights), coin_weights) )

  greater_than_one = tf.greater( coin_weights, np.float64(1) )
  unity_max_op = coin_weights.assign( tf.where (greater_than_one, tf.ones_like(coin_weights), coin_weights) )

  result_sum = tf.reduce_sum(coin_weights)
  unity_sum_op = coin_weights.assign(tf.divide(coin_weights, result_sum))
  
  constraints_op = tf.group(zero_minimum_op, unity_max_op, unity_sum_op)

  # Run
  learning_rate = 0.001
  steps = 15000
  
  init = tf.compat.v1.global_variables_initializer()

  # Training using Gradient Descent to minimize variance
  optimize_op = tf.compat.v1.train.GradientDescentOptimizer(learning_rate).minimize(portfolio_volatility)

  with tf.compat.v1.Session() as sess:
    sess.run(init)
    for i in range(steps):
      sess.run(optimize_op)
      sess.run(constraints_op)
      if i % 2500 == 0 :
        print("[round {:d}]".format(i))
        print("Weights", coin_weights.eval())
        print("Volatility: {:.2f}%".format(portfolio_volatility.eval() * 100))
        print("")
        
    sess.run(constraints_op)
    return coin_weights.eval()

weights = minimize_volatility()

pretty_weights2 = pd.DataFrame(weights * 100, index = coins, columns = ["Min Volatility Constraits Weight %"])
pretty_weights2

[round 0]
Weights [[0.11107754]
 [0.11106569]
 [0.11106784]
 [0.11107096]
 [0.11107564]
 [0.11104864]
 [0.11106226]
 [0.11106959]
 [0.11111168]]
Volatility: 3.89%

[round 2500]
Weights [[0.13266503]
 [0.06148583]
 [0.07345091]
 [0.09219096]
 [0.12011473]
 [0.        ]
 [0.04510483]
 [0.08242359]
 [0.34184704]]
Volatility: 2.44%

[round 5000]
Weights [[0.12348066]
 [0.        ]
 [0.        ]
 [0.03067048]
 [0.09347332]
 [0.        ]
 [0.        ]
 [0.00885073]
 [0.61279037]]
Volatility: 0.93%

[round 7500]
Weights [[0.07518953]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.03138857]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.87549581]]
Volatility: 0.38%

[round 10000]
Weights [[0.02670146]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.96790074]]
Volatility: 0.19%

[round 12500]
Weights [[1.53879414e-02]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [5.92686759e-04]
 [1.65715917e-03]
 [0.00000000e+00]
 

,Min Volatility Constraits Weight %
BTC,1.151921
ETH,0.000000
LTC,0.000000
XLM,0.000000
XRP,0.000000
ANT,0.146761
MANA,0.246624
TRX,0.000000
USDT,98.447421


Running zero_minimum_op triggers the assigment of 0 to coin_weights elements that are negative while unity_max_op does the same by replacing numbers above one, by 1. unity_sum_op ensures that the sum of all values equals 1.
Finally, constraints_op groups all three operations into one, and sess.run(constraints_op) triggers the group of operations.

Again, no surprise. USDT is a stablecoin, which is directly tied to the price of the US Dollar. It may have tiny fluctuations, but obviously the optimizer learns that the least volatility is achieved by putting all the eggs in the stablecoin basket.
Stablecoins allow investors to hold their money in a crypto wallet without the need to exchange into fiat currency. In scenarios of a bearish market, it obviously makes sense to sell into USDT and enter again when the trend reverses.
But if we look at cumulative_returns we can see that all the non-stablecoins have nice positive returns, and we don’t want to miss them. So, we just wrote our first portfolio optimizer, but for it to be useful we need to add returns into the equation, not just volatility.

### Sharpe ratio
The Sharpe Ratio is one of the most used metrics in the Modern Portfolio Theory. It combines both magnitudes we want into a simple formula:
Image for post
Where rp is the return of the portfolio, rf is the risk-free rate and sigma is the std deviation.
In crypto, the risk-free rate would correspond to the return of a stablecoin like USDT, but since its long-term average return is zero, our formula could be simplified into:
Image for post

So let’s get back to Colab and describe the Sharpe Ratio so we can optimize it:

In [179]:
# Optimize weights to maximize return/risk

import time
start = time.time()

def maximize_sharpe_ratio():
  
  # Define the model
  
  # 1) Variance
  
  coin_weights = tf.Variable(tf.random.uniform((len(coins), 1), dtype=tf.float64)) # our variables
  weighted_std_devs = tf.multiply(coin_weights, std_deviations)
  
  product_1 = tf.transpose(weighted_std_devs)
  product_2 = tf.matmul(product_1, correlation_matrix)
  
  portfolio_variance = tf.matmul(product_2, weighted_std_devs)
  portfolio_volatility = tf.sqrt(tf.reduce_sum(portfolio_variance))

  
  # 2) Return
  
  returns = np.full((len(coins), 1), 0.0) # same as coin_weights
  for coin_idx in range(0, len(coins)):
    returns[coin_idx] = cumulative_returns[coins[coin_idx]]
  
  portfolio_return = tf.reduce_sum(tf.multiply(coin_weights, returns))
  
  # 3) Return / Risk
  
  sharpe_ratio = tf.divide(portfolio_return, portfolio_volatility)
  
  # Constraints
  
  # all values positive, with unity sum
  weights_sum = tf.reduce_sum(coin_weights)
  constraints_op = coin_weights.assign(tf.divide(tf.abs(coin_weights), tf.abs(weights_sum) ))
  
  # Run
  #learning_rate = 0.0001
  learning_rate = 0.0015
  steps = 10000
  
  # Training using Gradient Descent to minimize cost
  optimize_op = tf.compat.v1.train.GradientDescentOptimizer(learning_rate, use_locking=True).minimize(tf.negative(sharpe_ratio))
  #2# 
  #optimize_op = tf.compat.v1.train.AdamOptimizer(learning_rate, use_locking=True).minimize(tf.negative(sharpe_ratio))
  #3# 
  #optimize_op = tf.compat.v1.train.AdamOptimizer(learning_rate=0.00005, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False).minimize(tf.negative(sharpe_ratio))
  #4# 
  #optimize_op = tf.compat.v1.train.AdagradOptimizer(learning_rate=0.01, initial_accumulator_value=0.1, use_locking=False).minimize(tf.negative(sharpe_ratio))
  
  
  init = tf.compat.v1.global_variables_initializer()
  
  with tf.compat.v1.Session() as sess:
    ratios = np.zeros(steps)
    returns = np.zeros(steps)
    sess.run(init)
    for i in range(steps):
      sess.run(optimize_op)
      sess.run(constraints_op)
      ratios[i] = sess.run(sharpe_ratio)
      returns[i] = sess.run(portfolio_return) * 100
      if i % 2000 == 0 : 
        sess.run(constraints_op)
        print("[round {:d}]".format(i))
        #print("Coin weights", sess.run(coin_weights))
        print("Volatility {:.2f} %".format(sess.run(portfolio_volatility)))
        print("Return {:.2f} %".format(sess.run(portfolio_return)*100))
        print("Sharpe ratio", sess.run(sharpe_ratio))
        print("")
    
    sess.run(constraints_op)
    # print("Coin weights", sess.run(coin_weights))
    print("Volatility {:.2f} %".format(sess.run(portfolio_volatility)))
    print("Return {:.2f} %".format(sess.run(portfolio_return)*100))
    print("Sharpe ratio", sess.run(sharpe_ratio))
    return sess.run(coin_weights)

weights = maximize_sharpe_ratio()

print("Took {:f}s to complete".format(time.time() - start))
pretty_weights3 = pd.DataFrame(weights * 100, index = coins, columns = ["Max Sharpe Weight %"])

pretty_weights3

[round 0]
Volatility 0.04 %
Return 115.98 %
Sharpe ratio 31.123457274434134

[round 2000]
Volatility 0.05 %
Return 334.04 %
Sharpe ratio 61.7936743653997

[round 4000]
Volatility 0.05 %
Return 334.04 %
Sharpe ratio 61.793675333416076

[round 6000]
Volatility 0.05 %
Return 334.04 %
Sharpe ratio 61.793675333593896

[round 8000]
Volatility 0.05 %
Return 334.04 %
Sharpe ratio 61.793675333593924

Volatility 0.05 %
Return 305.76 %
Sharpe ratio 59.68280624752567
Took 10.682679s to complete


,Max Sharpe Weight %
BTC,0.707438
ETH,0.044419
LTC,2.758155
XLM,2.544492
XRP,2.697571
ANT,5.324212
MANA,62.939349
TRX,1.985696
USDT,20.998668


In [191]:
p_weight=pd.concat([pretty_weights1["Min Volatility Non Constratits Weight %"],pretty_weights2["Min Volatility Constraits Weight %"],pretty_weights3["Max Sharpe Weight %"]],axis=1)
p_weight

,Min Volatility Non Constratits Weight %,Min Volatility Constraits Weight %,Max Sharpe Weight %
BTC,0.039343,1.151921,0.707438
ETH,0.017207,0.000000,0.044419
LTC,0.013672,0.000000,2.758155
XLM,0.017640,0.000000,2.544492
XRP,0.011229,0.000000,2.697571
ANT,0.031507,0.146761,5.324212
MANA,0.026585,0.246624,62.939349
TRX,0.015867,0.000000,1.985696
USDT,2.644630,98.447421,20.998668


### Wrap up
We just wrote a Cryptocurrency Portfolio Optimizer using Python, Tensorflow and Financial Risk Modeling. Writing an equivalent tool would have involved much time of research and development in the past.
The tool that we have explored allows to find optimal portfolios given the past performance of a set of assets. Machine Learning is not intented to find the absolute and exact answer for a problem. It is rather a very good tool to find close approximations in a very reasonable amount of time.
It is important to remember that a Portfolio Optimizer is just an assessment tool, not the “one true answer”. Every single result must be analyzed and interpreted by a human. We definitely need to check the price charts and market capitalization of each coin and do enough fundamental analysis (news, roadmap, team) before stepping into the ring.
Cryptocurrencies are wildly volatile compared to traditional securities. Do your own research and consider using this tool with more stable assets if your investment strategy is not aligned with such volatilities.